In [ ]:
# Databricks notebook source

# Exercice 01 : Manipulation de DataFrames Spark## Objectifs pedagogiquesA la fin de cet exercice, vous serez capable de :- Charger des donnees depuis une table Delta- Selectionner des colonnes specifiques- Filtrer des lignes selon des conditions- Trier des donnees- Renommer des colonnes- Creer de nouvelles colonnes calculees## Duree estimee : 45 minutes## Dataset : Gares SNCF

## PARTIE 1 : Concepts theoriques### Qu'est-ce qu'un DataFrame Spark ?Un DataFrame est une collection distribuee de donnees organisees en colonnes nommees.C'est equivalent a une table SQL ou un dataframe Pandas, mais optimise pour le traitement distribue.### Caracteristiques principales :- **Immutable** : Chaque transformation cree un nouveau DataFrame- **Lazy Evaluation** : Les transformations ne sont executees que quand une action est demandee- **Distribue** : Les donnees sont reparties sur plusieurs machines- **Optimise** : Le moteur Catalyst optimise automatiquement les requetes### Transformations vs Actions :**Transformations** (lazy - ne s'executent pas immediatement) :- `select()`, `filter()`, `withColumn()`, `orderBy()`, etc.**Actions** (executent les transformations) :- `show()`, `count()`, `collect()`, `write()`, etc.

## PARTIE 2 : Chargement des donnees

In [ ]:

print("EXERCICE 1.1 : Charger la table des gares")
print("=" * 70)

# Charger la table Silver des gares
df_gares = spark.table("gares_silver")

# Afficher le nombre de lignes et de colonnes
print(f"Nombre de lignes : {df_gares.count()}")
print(f"Nombre de colonnes : {len(df_gares.columns)}")

# Afficher les 5 premieres lignes
print("\nApercu des donnees :")
df_gares.show(5)


## PARTIE 3 : Selection de colonnes

In [ ]:

print("EXERCICE 1.2 : Selection de colonnes")
print("=" * 70)

# Methode 1 : Selectionner des colonnes specifiques
df_simple = df_gares.select("nom", "trigramme", "segment_clean", "code_departement")

print("Selection de 4 colonnes :")
df_simple.show(10)

# Methode 2 : Selectionner avec alias
from pyspark.sql.functions import col

df_alias = df_gares.select(
    col("nom").alias("nom_gare"),
    col("trigramme").alias("code"),
    col("categorie_gare").alias("categorie")
)

print("\nSelection avec alias :")
df_alias.show(10)


## EXERCICE PRATIQUE 1.3A VOUS : Selectionnez les colonnes suivantes avec des alias :- nom vers gare- code_departement vers dept- latitude vers lat- longitude vers lon

In [ ]:

# A COMPLETER
from pyspark.sql.functions import col

df_exercice = df_gares.select(
    # Completez ici
    col("nom").alias("gare"),
    col("code_departement").alias("dept"),
    col("latitude").alias("lat"),
    col("longitude").alias("lon")
)

df_exercice.show(10)


## PARTIE 4 : Filtrage de donnees

In [ ]:

print("EXERCICE 1.4 : Filtrage de lignes")
print("=" * 70)

# Methode 1 : Filtre simple
df_paris = df_gares.filter(col("code_departement") == "75")
print(f"Gares a Paris (75) : {df_paris.count()}")
df_paris.show(10)

# Methode 2 : Filtres multiples avec AND
df_principales_paris = df_gares.filter(
    (col("code_departement") == "75") &
    (col("segment_clean") == "A")
)
print(f"\nGares principales a Paris : {df_principales_paris.count()}")
df_principales_paris.show()

# Methode 3 : Filtres avec OR
df_grandes_villes = df_gares.filter(
    (col("code_departement") == "75") |
    (col("code_departement") == "69") |
    (col("code_departement") == "13")
)
print(f"\nGares a Paris, Lyon ou Marseille : {df_grandes_villes.count()}")

# Methode 4 : Utiliser isin() pour plusieurs valeurs
df_grandes_villes_2 = df_gares.filter(col("code_departement").isin("75", "69", "13"))
print(f"Meme resultat avec isin() : {df_grandes_villes_2.count()}")


## EXERCICE PRATIQUE 1.5A VOUS : Trouvez toutes les gares qui :- Sont de segment A (principales)- Ont "Paris" dans leur nom- Afficher uniquement : nom, trigramme, categorie_gare

In [ ]:

# A COMPLETER
from pyspark.sql.functions import col

df_exercice = df_gares.filter(
    # Completez les conditions ici
    (col("segment_clean") == "A") &
    (col("nom").like("%Paris%"))
).select("nom", "trigramme", "categorie_gare")

print(f"Nombre de gares trouvees : {df_exercice.count()}")
df_exercice.show()


## PARTIE 5 : Tri des donnees

In [ ]:

print("EXERCICE 1.6 : Tri des donnees")
print("=" * 70)

# Tri croissant
df_tri_asc = df_gares.orderBy("nom")
print("Tri croissant par nom :")
df_tri_asc.select("nom", "trigramme").show(10)

# Tri decroissant
df_tri_desc = df_gares.orderBy(col("nom").desc())
print("\nTri decroissant par nom :")
df_tri_desc.select("nom", "trigramme").show(10)

# Tri sur plusieurs colonnes
df_tri_multi = df_gares.orderBy(
    col("code_departement").asc(),
    col("nom").asc()
)
print("\nTri par departement puis par nom :")
df_tri_multi.select("code_departement", "nom", "trigramme").show(15)


## PARTIE 6 : Creation de nouvelles colonnes

In [ ]:

print("EXERCICE 1.7 : Creer de nouvelles colonnes")
print("=" * 70)

from pyspark.sql.functions import col, when, concat, lit, upper, length

# Methode 1 : Colonne simple calculee
df_avec_region = df_gares.withColumn(
    "est_ile_de_france",
    when(col("code_departement").isin("75", "77", "78", "91", "92", "93", "94", "95"), "Oui")
    .otherwise("Non")
)

print("Ajout d'une colonne est_ile_de_france :")
df_avec_region.select("nom", "code_departement", "est_ile_de_france").show(10)

# Methode 2 : Concatenation de colonnes
df_avec_libelle = df_gares.withColumn(
    "libelle_complet",
    concat(col("nom"), lit(" ("), col("trigramme"), lit(")"))
)

print("\nConcatenation de colonnes :")
df_avec_libelle.select("libelle_complet", "segment_clean").show(10)

# Methode 3 : Transformations de texte
df_avec_maj = df_gares.withColumn(
    "nom_majuscule",
    upper(col("nom"))
).withColumn(
    "longueur_nom",
    length(col("nom"))
)

print("\nTransformations de texte :")
df_avec_maj.select("nom", "nom_majuscule", "longueur_nom").show(10)


## EXERCICE PRATIQUE 1.8A VOUS : Creer les colonnes suivantes :1. **type_gare** :- "Majeure" si segment A ou B- "Locale" si segment C2. **identifiant** :- Concatener le code departement et le trigramme avec un tiret- Exemple : "75-MON" pour Montparnasse3. **nom_court** :- Si le nom fait plus de 20 caracteres, True, sinon False

In [ ]:

# A COMPLETER
from pyspark.sql.functions import col, when, concat, lit, length

df_exercice = df_gares \
    .withColumn(
        "type_gare",
        # Completez ici
        when(col("segment_clean").isin("A", "B"), "Majeure")
        .otherwise("Locale")
    ) \
    .withColumn(
        "identifiant",
        # Completez ici
        concat(col("code_departement"), lit("-"), col("trigramme"))
    ) \
    .withColumn(
        "nom_court",
        # Completez ici
        when(length(col("nom")) > 20, True).otherwise(False)
    )

# Afficher le resultat
df_exercice.select("nom", "type_gare", "identifiant", "nom_court").show(15)


## PARTIE 7 : Combinaison de toutes les operations

In [ ]:

print("EXERCICE 1.9 : Pipeline complete")
print("=" * 70)

# Exemple de pipeline complete combinant toutes les operations
df_final = df_gares \
    .filter(col("segment_clean").isNotNull()) \
    .withColumn(
        "region_approx",
        when(col("code_departement").isin("75", "77", "78", "91", "92", "93", "94", "95"), "Ile-de-France")
        .when(col("code_departement").isin("69", "01", "42", "38", "73", "74"), "Auvergne-Rhone-Alpes")
        .when(col("code_departement").isin("13", "83", "84", "04", "05", "06"), "PACA")
        .otherwise("Autre")
    ) \
    .withColumn(
        "importance",
        when(col("segment_clean") == "A", 3)
        .when(col("segment_clean") == "B", 2)
        .otherwise(1)
    ) \
    .select(
        "nom",
        "trigramme",
        "code_departement",
        "region_approx",
        "categorie_gare",
        "importance"
    ) \
    .orderBy(col("importance").desc(), col("nom").asc())

print("Pipeline complete executee :")
df_final.show(20)

print(f"\nNombre total de gares traitees : {df_final.count()}")


## EXERCICE FINAL 1.10 : Projet completCreez une pipeline complete qui :1. Charge la table gares_silver2. Filtre uniquement les gares des segments A et B3. Ajoute une colonne "zone_geo" :- "Nord" si code dept commence par 0, 5, 59, 62, 80- "Sud" si code dept commence par 1, 2, 3, 4, 6, 83, 84- "Autre" sinon4. Ajoute une colonne "a_coordonnees" : True si latitude ET longitude non nulles5. Selectionne : nom, trigramme, zone_geo, a_coordonnees, categorie_gare6. Trie par zone_geo puis nom7. Affiche les 30 premieres lignes8. Compte combien de gares par zone_geo

In [ ]:

# A COMPLETER - SOLUTION PROPOSEE
from pyspark.sql.functions import col, when

# Pipeline complete
df_projet = df_gares \
    .filter(col("segment_clean").isin("A", "B")) \
    .withColumn(
        "zone_geo",
        when(col("code_departement").isin("02", "08", "51", "59", "62", "80"), "Nord")
        .when(col("code_departement").isin("13", "30", "34", "66", "83", "84"), "Sud")
        .otherwise("Autre")
    ) \
    .withColumn(
        "a_coordonnees",
        when((col("latitude").isNotNull()) & (col("longitude").isNotNull()), True)
        .otherwise(False)
    ) \
    .select("nom", "trigramme", "zone_geo", "a_coordonnees", "categorie_gare") \
    .orderBy("zone_geo", "nom")

print("Resultat de la pipeline :")
df_projet.show(30)

print("\nRepartition par zone geographique :")
df_projet.groupBy("zone_geo").count().orderBy("count", ascending=False).show()


## PARTIE 8 : Sauvegarde du resultat

In [ ]:

print("EXERCICE 1.11 : Sauvegarder le resultat")
print("=" * 70)

# Sauvegarder le DataFrame final en table Delta
df_projet.write.format("delta").mode("overwrite").saveAsTable("gares_exercice_01")

print("Table 'gares_exercice_01' creee avec succes !")

# Verifier la table
df_verification = spark.table("gares_exercice_01")
print(f"\nVerification : {df_verification.count()} lignes sauvegardees")


## RESUME DES CONCEPTS APPRISDans cet exercice, vous avez appris :1. **Charger des donnees** : `spark.table()`2. **Selectionner des colonnes** : `select()`, `alias()`3. **Filtrer des lignes** : `filter()`, `isin()`, `like()`4. **Trier des donnees** : `orderBy()`, `asc()`, `desc()`5. **Creer des colonnes** : `withColumn()`, `when()`, `otherwise()`6. **Transformer du texte** : `concat()`, `upper()`, `length()`7. **Enchainer des operations** : Pipeline avec `\`8. **Sauvegarder** : `write.saveAsTable()`## Prochaine etapePassez a l'**Exercice 02 : Transformations et agregations** pour apprendre :- Les fonctions d'agregation (sum, avg, count, etc.)- Les regroupements (groupBy)- Les jointures entre tables- Les fonctions de date et heure